In [17]:
!pip install keras

     ---------------------------------------- 0.0/46.1 kB ? eta -:--:--
     -------- ------------------------------- 10.2/46.1 kB ? eta -:--:--
     ---------------- --------------------- 20.5/46.1 kB 217.9 kB/s eta 0:00:01
     ---------------- --------------------- 20.5/46.1 kB 217.9 kB/s eta 0:00:01
     ------------------------- ------------ 30.7/46.1 kB 163.8 kB/s eta 0:00:01
     ------------------------- ------------ 30.7/46.1 kB 163.8 kB/s eta 0:00:01
     -------------------------------------- 46.1/46.1 kB 153.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.1 MB 660.6 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.1 MB 656.4 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.1 MB 653.6 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.1 MB 653.6 kB/s eta 0:00:02
   -- ------------------------------------- 0.1/1.1 MB 653.6 kB/s eta 0:00

In [25]:
!pip install tensorflow

^C


In [50]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [51]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, GRU, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [57]:
data = pd.read_csv('urdu-sentiment-corpus-v1.tsv', sep='\t', header=None, names=['Tweet', 'Class'])

In [58]:
data.head()

,Tweet,Class
0,Tweet,Class
1,میں نے ایٹم بم بنایا ھے ۔۔۔۔او بھائی ایٹم بمب ...,P
2,چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...,N
3,ٹویٹر کا خیال کیسے آیا ؟,O
4,"سرچ انجن گوگل کے نائب صدر نے فضا میں ، 130,000...",P


In [59]:
data.isnull().sum()

Tweet    0
Class    1
dtype: int64

In [60]:
data = data[data['Class'].isin(['P', 'N'])] 
data.isnull().sum()

Tweet    0
Class    0
dtype: int64

In [61]:
data.shape
print(data['Class'].unique())

['P' 'N']


In [62]:
data.columns

Index(['Tweet', 'Class'], dtype='object')

In [63]:
def preprocess_text(text, stop_words):
    text = re.sub(r'[^\w\s]', '', text) 
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'\d+', '', text)
    tokens = text.split()
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)
urdu_stopwords = ['کا', 'کی', 'ہے', 'میں',"اور", "کے", "میں", "کی", "ہے", "یہ", "پر", "سے", "ہیں", "کو","ہو","وہ", "اس", "کا", "جو", "کر", "تھا", "تو", "ہوں", "گی"]

# Preprocess the dataset 
data['Preprocessed_Tweet'] = data['Tweet'].apply(lambda x: preprocess_text(x, urdu_stopwords))
label_mapping = {'P': 1, 'N': 0}
data['Class'] = data['Class'].map(label_mapping)
texts = data['Preprocessed_Tweet'].values
labels = data['Class'].values
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_sequence_length = max(len(x) for x in sequences)
data_padded = pad_sequences(sequences, maxlen=max_sequence_length)

In [64]:
print(data_padded)

[[   0    0    0 ...  715 1822 1823]
 [   0    0    0 ...    2   60  217]
 [   0    0    0 ...  305  718 1038]
 ...
 [   0    0    0 ...  825 4991  648]
 [   0    0    0 ...  633  268  634]
 [   0    0    0 ...   65  503  712]]


In [65]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, GRU, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

def create_model(model_type, num_layers, dropout_rate, vocab_size, max_sequence_length):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=128, input_shape=(max_sequence_length,)))
    
    if model_type == 'RNN':
        for i in range(num_layers - 1):
            model.add(SimpleRNN(128, return_sequences=True, dropout=dropout_rate))
        model.add(SimpleRNN(128, dropout=dropout_rate))
    elif model_type == 'GRU':
        for i in range(num_layers - 1):
            model.add(GRU(128, return_sequences=True, dropout=dropout_rate))
        model.add(GRU(128, dropout=dropout_rate))
    elif model_type == 'LSTM':
        for i in range(num_layers - 1):
            model.add(LSTM(128, return_sequences=True, dropout=dropout_rate))
        model.add(LSTM(128, dropout=dropout_rate))
    elif model_type == 'BiLSTM':
        for i in range(num_layers - 1):
            model.add(Bidirectional(LSTM(128, return_sequences=True, dropout=dropout_rate)))
        model.add(Bidirectional(LSTM(128, dropout=dropout_rate)))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [66]:
vocab_size = len(tokenizer.word_index) + 1
max_sequence_length = max(len(x) for x in data_padded)

X_train, X_test, y_train, y_test = train_test_split(data_padded, labels, test_size=0.25, random_state=42)
model_types = ['RNN', 'GRU', 'LSTM', 'BiLSTM']
num_layers_options = [2, 3]
dropout_rate_options = [0.3, 0.7]
results = []

for model_type in model_types:
    for num_layers in num_layers_options:
        for dropout_rate in dropout_rate_options:
            model = create_model(model_type, num_layers, dropout_rate, vocab_size, max_sequence_length)
            print(f"Training {model_type} with {num_layers} layers and dropout rate of {dropout_rate}")
            history = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test), verbose=2)
            
            # Evaluation
            y_pred = model.predict(X_test)
            y_pred_classes = np.round(y_pred).astype(int)
            report = classification_report(y_test, y_pred_classes, target_names=['Negative', 'Positive'], output_dict=True)
            accuracy = report['accuracy']
            
            # Save results
            results.append({
                'Model Type': model_type,
                'Number of Layers': num_layers,
                'Dropout Rate': dropout_rate,
                'Accuracy': accuracy,
                'Precision (Positive)': report['Positive']['precision'],
                'Recall (Positive)': report['Positive']['recall'],
                'F1-Score (Positive)': report['Positive']['f1-score']
            })

E:\Coding_Softwares\Lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training RNN with 2 layers and dropout rate of 0.3
Epoch 1/10
6/6 - 11s - 2s/step - accuracy: 0.5109 - loss: 0.7234 - val_accuracy: 0.5061 - val_loss: 0.6915
Epoch 2/10
6/6 - 1s - 91ms/step - accuracy: 0.7371 - loss: 0.5879 - val_accuracy: 0.5061 - val_loss: 0.7078
Epoch 3/10
6/6 - 1s - 91ms/step - accuracy: 0.8896 - loss: 0.3857 - val_accuracy: 0.5306 - val_loss: 0.8174
Epoch 4/10
6/6 - 1s - 91ms/step - accuracy: 0.9523 - loss: 0.1639 - val_accuracy: 0.5143 - val_loss: 1.0819
Epoch 5/10
6/6 - 1s - 91ms/step - accuracy: 0.9605 - loss: 0.0852 - val_accuracy: 0.5224 - val_loss: 1.2086
Epoch 6/10
6/6 - 1s - 92ms/step - accuracy: 0.9891 - loss: 0.0423 - val_accuracy: 0.5143 - val_loss: 1.2352
Epoch 7/10
6/6 - 1s - 89ms/step - accuracy: 0.9864 - loss: 0.0337 - val_accuracy: 0.5673 - val_loss: 1.1745
Epoch 8/10
6/6 - 1s - 90ms/step - accuracy: 0.9918 - loss: 0.0219 - val_accuracy: 0.5469 - val_loss: 1.2119
Epoch 9/10
6/6 - 1s - 84ms/step - accuracy: 0.9986 - loss: 0.0160 - val_accuracy: 0.56

E:\Coding_Softwares\Lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 - 11s - 2s/step - accuracy: 0.4973 - loss: 0.7433 - val_accuracy: 0.4694 - val_loss: 0.8326
Epoch 2/10
6/6 - 1s - 98ms/step - accuracy: 0.5436 - loss: 0.7062 - val_accuracy: 0.5510 - val_loss: 0.6899
Epoch 3/10
6/6 - 1s - 93ms/step - accuracy: 0.5804 - loss: 0.6812 - val_accuracy: 0.4857 - val_loss: 0.7018
Epoch 4/10
6/6 - 1s - 94ms/step - accuracy: 0.6076 - loss: 0.6476 - val_accuracy: 0.5592 - val_loss: 0.6880
Epoch 5/10
6/6 - 1s - 98ms/step - accuracy: 0.7030 - loss: 0.5937 - val_accuracy: 0.5265 - val_loss: 0.7238
Epoch 6/10
6/6 - 1s - 90ms/step - accuracy: 0.7943 - loss: 0.4724 - val_accuracy: 0.5184 - val_loss: 0.7580
Epoch 7/10
6/6 - 1s - 93ms/step - accuracy: 0.8542 - loss: 0.3352 - val_accuracy: 0.5388 - val_loss: 0.9470
Epoch 8/10
6/6 - 1s - 94ms/step - accuracy: 0.8937 - loss: 0.2656 - val_accuracy: 0.5224 - val_loss: 0.9754
Epoch 9/10
6/6 - 1s - 110ms/step - accuracy: 0.9223 - loss: 0.1939 - val_accuracy: 0.5265 - val_loss: 1.0857
Epoch 10/10
6/6 - 1s - 91ms/step - accu

E:\Coding_Softwares\Lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

Training BiLSTM with 2 layers and dropout rate of 0.7
Epoch 1/10
6/6 - 21s - 4s/step - accuracy: 0.4578 - loss: 0.6976 - val_accuracy: 0.4694 - val_loss: 0.6955
Epoch 2/10
6/6 - 3s - 564ms/step - accuracy: 0.5286 - loss: 0.6906 - val_accuracy: 0.5306 - val_loss: 0.6917
Epoch 3/10
6/6 - 4s - 598ms/step - accuracy: 0.5136 - loss: 0.6928 - val_accuracy: 0.5388 - val_loss: 0.6909
Epoch 4/10
6/6 - 3s - 470ms/step - accuracy: 0.5681 - loss: 0.6855 - val_accuracy: 0.5143 - val_loss: 0.6920
Epoch 5/10
6/6 - 3s - 444ms/step - accuracy: 0.6458 - loss: 0.6687 - val_accuracy: 0.5837 - val_loss: 0.6805
Epoch 6/10
6/6 - 3s - 448ms/step - accuracy: 0.6839 - loss: 0.6156 - val_accuracy: 0.5796 - val_loss: 0.6671
Epoch 7/10
6/6 - 3s - 451ms/step - accuracy: 0.8406 - loss: 0.4205 - val_accuracy: 0.6327 - val_loss: 0.8494
Epoch 8/10
6/6 - 3s - 473ms/step - accuracy: 0.9169 - loss: 0.2114 - val_accuracy: 0.5918 - val_loss: 1.0364
Epoch 9/10
6/6 - 3s - 482ms/step - accuracy: 0.9537 - loss: 0.1418 - val_acc

E:\Coding_Softwares\Lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training BiLSTM with 3 layers and dropout rate of 0.3
Epoch 1/10
6/6 - 30s - 5s/step - accuracy: 0.4905 - loss: 0.6940 - val_accuracy: 0.4653 - val_loss: 0.6951
Epoch 2/10
6/6 - 5s - 833ms/step - accuracy: 0.5341 - loss: 0.6912 - val_accuracy: 0.5551 - val_loss: 0.6909
Epoch 3/10
6/6 - 5s - 783ms/step - accuracy: 0.6281 - loss: 0.6737 - val_accuracy: 0.6000 - val_loss: 0.6791
Epoch 4/10
6/6 - 5s - 769ms/step - accuracy: 0.8065 - loss: 0.5664 - val_accuracy: 0.6327 - val_loss: 0.6893
Epoch 5/10
6/6 - 5s - 800ms/step - accuracy: 0.9278 - loss: 0.2480 - val_accuracy: 0.6449 - val_loss: 0.9401
Epoch 6/10
6/6 - 5s - 835ms/step - accuracy: 0.9714 - loss: 0.0867 - val_accuracy: 0.6245 - val_loss: 1.4653
Epoch 7/10
6/6 - 5s - 752ms/step - accuracy: 0.9809 - loss: 0.0632 - val_accuracy: 0.6449 - val_loss: 1.6797
Epoch 8/10
6/6 - 5s - 796ms/step - accuracy: 0.9946 - loss: 0.0260 - val_accuracy: 0.6245 - val_loss: 1.6168
Epoch 9/10
6/6 - 5s - 850ms/step - accuracy: 0.9986 - loss: 0.0111 - val_acc

E:\Coding_Softwares\Lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training BiLSTM with 3 layers and dropout rate of 0.7
Epoch 1/10
6/6 - 31s - 5s/step - accuracy: 0.4986 - loss: 0.6940 - val_accuracy: 0.4653 - val_loss: 0.6942
Epoch 2/10
6/6 - 5s - 843ms/step - accuracy: 0.5014 - loss: 0.6938 - val_accuracy: 0.4735 - val_loss: 0.6932
Epoch 3/10
6/6 - 5s - 858ms/step - accuracy: 0.5123 - loss: 0.6967 - val_accuracy: 0.5306 - val_loss: 0.6918
Epoch 4/10
6/6 - 5s - 847ms/step - accuracy: 0.5136 - loss: 0.6934 - val_accuracy: 0.5347 - val_loss: 0.6911
Epoch 5/10
6/6 - 5s - 874ms/step - accuracy: 0.5572 - loss: 0.6848 - val_accuracy: 0.5592 - val_loss: 0.6908
Epoch 6/10
6/6 - 10s - 2s/step - accuracy: 0.5504 - loss: 0.6840 - val_accuracy: 0.6082 - val_loss: 0.6864
Epoch 7/10
6/6 - 5s - 833ms/step - accuracy: 0.6390 - loss: 0.6586 - val_accuracy: 0.5265 - val_loss: 0.6785
Epoch 8/10
6/6 - 5s - 901ms/step - accuracy: 0.7834 - loss: 0.4801 - val_accuracy: 0.5429 - val_loss: 1.0577
Epoch 9/10
6/6 - 5s - 883ms/step - accuracy: 0.8610 - loss: 0.3574 - val_accur

In [68]:
results_df = pd.DataFrame(results)

In [72]:
results_df.head(16)

,Model Type,Number of Layers,Dropout Rate,Accuracy,Precision (Positive),Recall (Positive),F1-Score (Positive)
0,RNN,2,0.3,0.567347,0.545455,0.469565,0.504673
1,RNN,2,0.7,0.526531,0.496241,0.573913,0.532258
2,RNN,3,0.3,0.510204,0.478992,0.495652,0.487179
3,RNN,3,0.7,0.510204,0.461538,0.260870,0.333333
4,GRU,2,0.3,0.595918,0.571429,0.556522,0.563877
5,GRU,2,0.7,0.608163,0.600000,0.495652,0.542857
6,GRU,3,0.3,0.567347,0.537815,0.556522,0.547009
7,GRU,3,0.7,0.600000,0.558621,0.704348,0.623077
8,LSTM,2,0.3,0.624490,0.587786,0.669565,0.626016
9,LSTM,2,0.7,0.579592,0.538961,0.721739,0.617100
